# Math 5031
## Home Work  2
### MingxuanJiang 5588030
**Due:  Monday, September 28




Collarborated with HaoChen and JingyiLi

1. Create a new bootstrapping function with the same inputs and outputs as from the first homework, but this time implement the constant forward assumption.  

In [1]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import fsolve

In [106]:
mats = [1,2,3]
par_yields = [0.01,0.02,0.03]

In [87]:
def BootCon(mats,par_yields):
        
    # the coupon matrix at first mats
    C1 = np.kron(np.ones(int(mats[0]*2)), par_yields[0])/2
    C1[-1] += 1
    Power_index = np.arange(1,len(C1)+1).reshape(-1,1)
    # Get the Zcb at the time 0.5 
    Zcb = [fsolve(lambda x: np.dot(C1, np.power(np.ones((len(C1),1))*x,Power_index))-1, 2)[0]]
    # Get the rest Zcb before mats[1]
    k=1
    for i in np.arange(0.5,mats[0],0.5):
        k+=1
        Zcb.append(np.power(Zcb[0],k))
    
    # the rest of mats
    for index in range(1,len(mats)):
        # the coupon matrix
        C = np.kron(np.ones(int(mats[index]*2)), par_yields[index])/2
        C[-1] += 1
        Power_index1 = np.zeros(len(Zcb))
        Power_index2 = np.arange(1,(mats[index]-mats[index-1])*2+1)
        Power_index = np.hstack((Power_index1,Power_index2))
        while len(Zcb) < int(mats[index]/0.5):
            Zcb.append(Zcb[-1])
            
        z = fsolve(lambda x: np.dot(C, np.multiply(Zcb, np.power(np.ones(len(Zcb))*x,Power_index)))-1,0)[0]
        #print(z)
        k = 1
        for i in np.arange(mats[index-1]*2,mats[index]*2,1):
            Zcb[int(i)] *= np.power(z,k)
            k += 1
    
    time = np.arange(0.5,mats[-1]+0.1,0.5)
    SpotRate = -np.log(Zcb)/time
    
    Zcb = np.array(Zcb)
    return Zcb,SpotRate



In [107]:
# test
Zcb,SpotRate = BootCon(mats,par_yields)
print('zcb',Zcb)
print('spot rate',SpotRate)

zcb [0.99502488 0.9900745  0.97532129 0.96078791 0.93680888 0.91342831]
spot rate [0.00997508 0.00997508 0.01665889 0.02000079 0.02611039 0.03018346]


In [89]:
print(sum(Zcb*np.array([1.5,1.5,1.5,1.5,1.5,101.5])))

99.99999999999999


2. Consider the following par-yield curve as of 12/31/2020:

|Term|BEY%|
|---|---|
|1|1.51|
|2|1.78|
|3|2.11|
|4|2.75|
|5|2.94|
|7|3.45|
|10|3.87|
|20|4.00|
|30|3.50|


     Using the bootstrapping function from ex. 1 or from Home work 1, value the following bonds as of 12/31/2020:

   a) A 17-year bond with $5.00\% BEY$ coupons

   b) A 6-year bond with $1.00\% BEY$ coupons

   c) A 5-year floating rate bond with coupon rate $c(t) =  y_2(t-0.5) + .002$

   d) A 10-year inverse floater where the coupon rate $c(t) = .20 - y_2(t-0.5)$

In [90]:
def simplebootstrap(mats, par_yields):
    
    
    #  This is not always needed, but sometimes helpful
    mats = np.array(mats)
    mats = mats.flatten()
    par_yields = np.array(par_yields)
    par_yields = par_yields.flatten()  
    #
    
    
    firstmat = mats[0]
    lastmat = mats[-1]  #mats should be fed in in order.
    
    times = np.arange(0.5, lastmat+.5, 0.5)
    
    
    if mats[0] > 0.5:  #in case first par yield is not 6 months
        
        earlymats = np.arange(0.5,firstmat,.5)
        
        mats = np.append(earlymats,mats)
        
        earlypars = np.ones((1,2*firstmat -1)) * par_yields[0]
        earlypars = earlypars.flatten()
        par_yields = np.append(earlypars, par_yields)
        
    
    bey_interp = np.interp(times,mats,par_yields)
    
    # Make the coupon matrix
    
    C = np.kron(np.ones((len(bey_interp),1)),bey_interp).T /2
    
    
    C = np.tril(C) + np.eye(len(bey_interp))
    
    
    pars = np.ones((len(bey_interp),1))
    # solve matrix equation
    
    ZCB = np.matmul(np.linalg.inv(C),pars).flatten()
    
    
    spots = -np.log(ZCB) *  1/times
    
    return ZCB, spots

In [91]:
terms = np.array([1,2,3,4,5,7,10,20,30])
BEYs = np.array([.0151,.0178,.0211,.0275,.0294,.0345,.0387,.04,.035])
Z,rates = simplebootstrap(terms,BEYs)

a)

In [92]:
cfs1 = 0.05/2*np.ones((17*2,1)).flatten()
cfs1[-1] += 1
print(cfs1)
value1 = np.inner(cfs1,Z[0:17*2])
print('The value is',round(100*value1,2))

[0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025
 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025
 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 1.025]
The value is 113.01


b)

In [93]:
cfs2 = 0.01/2*np.ones((6*2,1)).flatten()
cfs2[-1] += 1
print(cfs2)
value2 = np.inner(cfs2,Z[0:6*2])
print('The value is',round(100*value2,2))

[0.005 0.005 0.005 0.005 0.005 0.005 0.005 0.005 0.005 0.005 0.005 1.005]
The value is 87.89


c)$$B(0) = B_{standard}(0) + B_{spreadonly}(0)$$
$$B(0) = 1+s/2A_2(0,T)$$

In [94]:
value3 = (sum(0.002/2*Z[0:10])+1)
print('The value is',round(100*value3,2))

The value is 100.94


d)$$B_{inverse}(0,T) = Z(0,T) + s/2A_2(0,T) - B_{FCO}(0,T)$$
$$B_{FCO}(0,T) = 100*(1-Z(0,T))$$

In [95]:
value4 = Z[20-1]+sum(0.2/2*Z[0:20])-(1-Z[20-1])
print('The value is',round(100*value4,2))

The value is 203.78


3. 

 i) Calculate the duration and convexity of bonds a and b in question 2

 ii) Estimate the impact of a $0.50\%$ increase in rates on the value of each bond using duration and then duration and convexity.  Compare each to the exact answer.

i)

In [96]:
# Duration
times1 = np.arange(0.5,17.1,0.5)
Dur1 = np.inner(Z[0:17*2], times1 * cfs1)/ value1
print('The duration of bond a is',Dur1)
times2 = np.arange(0.5,6.1,0.5)
Dur2 = np.inner(Z[0:6*2], times2 * cfs2) / value2
print('The duration of bond b is',Dur2)

The duration of bond a is 11.855981033468666
The duration of bond b is 5.820950307303775


In [97]:
# Convexity
cv1 = np.inner(Z[0:17*2],cfs1*times1**2)/value1
print('The convexity of bond a is', cv1)
cv2 = np.inner(Z[0:6*2],cfs2*times2**2)/value2
print('The convexity of bond b is', cv2)

The convexity of bond a is 174.78951267560595
The convexity of bond b is 34.54736172329511


ii) the impact of a 0.5% increase in rates

In [98]:
print('bond a')
decrease_percent1 = 0.5*Dur1
est_loss1 = decrease_percent1/100*value1
z_new1 = np.exp(-(rates[0:17*2]+0.005)*times1)
value_new1 = np.inner(cfs1,z_new1)
decrease_percent_exact = (1 - value_new1/value1)*100
print('decrease percent',decrease_percent_exact)

loss_exact = value1 - value_new1
print('loss',loss_exact*100)

bond a
decrease percent 5.715140969577092
loss 6.458492574594166


In [99]:
impact1 = -0.005*Dur1+1/2*0.005*0.005*cv1
print('loss',impact1*value1*100)

loss -6.452121998165963


In [100]:
print('bond b')
decrease_percent2 = 0.5*Dur2
est_loss2 = decrease_percent2/100*value2
z_new2 = np.exp(-(rates[0:12]+0.005)*times2)
value_new2 = np.inner(cfs2,z_new2)
decrease_percent_exact = (1 - value_new2/value2)*100
print('decrease percent',decrease_percent_exact)

loss_exact = value2 - value_new2
print('loss',loss_exact*100)

bond b
decrease percent 2.867717283528315
loss 2.5205118418146433


In [101]:
impact2 = -0.005*Dur2+1/2*0.005*0.005*cv2
print('loss',impact2*value2*100)

loss -2.520137127425014


4.  Three semi-annual coupon  bonds as given in the table.

|Bond|Coupon Rate|Maturity|Duration|Convexity|
|---|---|---|---|---|
|X|$2.00\%$|3|2.7|6|
|Y|$5.00\%$|7|5.1|40|
|Z|$4.00\%$|20|12.0|180|

A  pension plan has plan liabilities worth $\$90$M and currently $Dur_L = 11$ and $Conv_L = 100$.  The assets are currently worth $\$100$M and are invested $50/50$ into bonds $X$ and $Y$.  In order to protect against changes in interest rates, recommend a change in the portfolio allocation and explain your recommendation.

In [2]:
Weight = np.array([50,50,0])
Dur = np.array([2.7,5.1,12])
Con = np.array([6,40,180])
DurPI = np.inner(Weight,Dur)/sum(Weight)
print('Initial portfolio duration is',DurPI)
ConPI = np.inner(Weight,Con)/sum(Weight)
print('Initial portfolio convexity is',ConPI)

Initial portfolio duration is 3.9
Initial portfolio convexity is 23.0


$$2.7x+5.1y+12z=11*90/100$$
$$6x+40y+180z\geq90*100$$
$$x+y+z=100$$

In [7]:
A = np.array([[2.7,5.1,12],[6,40,180],[1,1,1]])
B = np.array([990,9000,100])
w = np.matmul(np.linalg.inv(A),B)
print('Weight of bond X is',w[0],'. Weight of bond Y is',w[1],'. Weight of bond Z is',w[2])

Weight of bond X is -322.48520710059256 . Weight of bond Y is 465.0887573964508 . Weight of bond Z is -42.6035502958582


We can't short bond. So assume weight of bond X is 0.

In [9]:
A = np.array([[1,1],[5.1,12]])
B = np.array([100,990])
w = np.matmul(np.linalg.inv(A),B)
print('Weight of bond X is',0,'. Weight of bond Y is',w[0],'. Weight of bond Z is',w[1])

Weight of bond X is 0 . Weight of bond Y is 30.434782608695684 . Weight of bond Z is 69.56521739130433


In [10]:
print('The dollar convexity is',np.inner(w,np.array([40,180])))

The dollar convexity is 13739.130434782608


In this case, A > L. I recommend to change the portfilio invested 0/0.30/0.69 into bonds X, Y and Z. This is because we need the Duration equal to $Dur_L = Dur_A*L/A$. Meanwhile, the sum of weight is equal to 1 and the convexity is greater than 9000.

It means that we need to change the weight of bond X to 0 from 0.5, the weight of bond Y to 0.30 from 0.5 and the weight of bond Z to 0.69 from 0 to get our portfolio.